# Direct methods for solving linear systems (homework)


**Exercise 1.** Let us consider the linear system $A\mathbf{x} = \mathbf{b}$ where
$$
  A = 
  \begin{bmatrix}
  \epsilon & 1 & 2\\
  1 & 3 & 1 \\
  2 & 1 & 3 \\
  \end{bmatrix}.
$$

1. Find the range of values of $\epsilon \in \mathbb{R}$ such that the matrix $A$ is symmetric and positive definite.
**Suggestion**: use the *Sylvester's criterion* which states that  a symmetric matrix $A \in \mathbb{R}^{n \times n}$ is positive definite if and only if all the main minors (The main minors of $A \in \mathbb{R}^{n \times n}$ are the determinants of the submatrices $A_p = (a_{i,j})_{1 \leq i, j \leq p}$, $p = 1, ..., n$). of $A$ are positive.
2. What factorization is more suitable for solving the linear system $A\mathbf{x}=\mathbf{b}$ for the case $\epsilon=0$? Motivate the answer.
3. Compute the Cholesky factorization $A = R^T R$ for the case $\epsilon = 2$.
4. Given $\mathbf{b} = (1,1,1)^T$, solve the linear system by using the Cholesky factorization computed at the previous point.



**Exercise 2.** Let us consider the following matrix $A \in \mathbb R^{3 \times 3}$ depending on the parameter $\epsilon \in \mathbb R$:
$$
A =
\begin{bmatrix}
1 & \epsilon & -1 \\
\epsilon & \frac{35}3 & 1 \\
-1 & \epsilon & 2 \\
\end{bmatrix}.
$$



1. Calculate the values of the parameter $\epsilon \in \mathbb R$ for which the matrix $A$ is invertible (non singular).

2. Calculate the Gauss factorization $LU$ of the matrix $A$ (when non singular) for a generic value of the parameter $\epsilon \in \mathbb R$.

3. Calculate the values of the parameter $\epsilon \in \mathbb R$ for which the Gauss factorization $LU$ of the matrix $A$  (when non singular) exists and is unique.

4. Set $\epsilon = \sqrt{\frac{35}3}$ and use the pivoting technique to calculate the Gauss factorization $LU$ of the matrix $A$.

5. For $\epsilon=1$, the matrix $A$ is symmetric and positive definite. Calculate the corresponding Cholesky factorization of the matrix $A$, i.e. the upper triangular matrix with positive elements on the diagonal, say $R$, for which $A = R^T R$.

**Exercise 1.**

In [1]:
%matplotlib inline
from numpy import *
from sympy import *
eps = Symbol('eps')
A=[[eps, 1, 2], [1, 3, 1], [2, 1, 3]]
A = Matrix(A)
p=S.Reals
for i in range(1,A.rows+1):
    a=A[0:i,0:i]
    q=solve_poly_inequality(Poly(a.det()),">")
    p=p.intersect(q)
print("1) A is symmetric and positive definite for epsilon in: ",p)

('1) A is symmetric and positive definite for epsilon in: ', Interval.open(11/8, oo))


2) For for the case $\epsilon=0$ A is not symmetric and positive definite, for solving the linear system $A\mathbf{x}=\mathbf{b}$ Gauss factorization LU is more suitable because Cholesky factorization can not be used.

In [2]:
A=matrix([[2.,1.,2.],[1.,3.,1.],[2.,1.,3.]])

def cholesky(A):
    A = A.copy()
    N = len(A)
    for k in range(N-1):
        A[k,k] = sqrt(A[k,k])
        A[k+1:N,k] = A[k+1:N,k]/A[k,k]
        
        for j in range(k+1,N):
            A[j:N,j] = A[j:N,j] - A[j:N,k]*A[j,k]
    
    A[-1,-1] = sqrt(A[-1,-1])
    L=tril(A)
    return L.T

print("3) Cholesky factorization, Matrix A is:")
print(A)
R=cholesky(A)
print("Matrix R is: ")
print(R)
print("A = R.T*R = ", dot(R.T,R))


3) Cholesky factorization, Matrix A is:
[[ 2.  1.  2.]
 [ 1.  3.  1.]
 [ 2.  1.  3.]]
Matrix R is: 
[[  1.41421356e+00   7.07106781e-01   1.41421356e+00]
 [  0.00000000e+00   1.58113883e+00  -1.40433339e-16]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]
('A = R.T*R = ', array([[ 2.,  1.,  2.],
       [ 1.,  3.,  1.],
       [ 2.,  1.,  3.]]))


In [3]:
def L_solve(L,rhs):
    x = zeros_like(rhs)
    N = len(L)
        
    x[0] = rhs[0]/L[0,0]
    for i in range(1,N):
        x[i] = (rhs[i] - dot(L[i, 0:i], x[0:i]))/L[i,i]
    
    return x

def U_solve(U,rhs):
    x = zeros_like(rhs)
    N=len(U)
            
    x[-1] = rhs[-1]/U[-1,-1]
    for i in reversed(range(N-1)):
        x[i] = (rhs[i] -dot(U[i, i+1 : N-1], x[i+1 : N-1]))/U[i,i]
        
    return x

b=[1.,1.,1.]
y = L_solve(R.T,b)
u = U_solve(R,y)
print("4) Solve Ax=b, with b =", b)
print("Solution is u=",u)
print("Au = ",dot(A,u))

('4) Solve Ax=b, with b =', [1.0, 1.0, 1.0])
('Solution is u=', array([  4.00000000e-01,   2.00000000e-01,  -2.22044605e-16]))
('Au = ', matrix([[ 1.,  1.,  1.]]))


**Exercise 2.**

In [4]:
eps = Symbol('eps')
A=[[1, eps, -1], [eps, 35./3, 1], [-1, eps, 2]]
A = Matrix(A)
print("1) A invertible for epsilon different than: ", solve(A.det()))

('1) A invertible for epsilon different than: ', [-2.33333333333333, 1.66666666666667])


In [5]:
def LU(A):
    A = A[:,:]
    N=A.rows
    for k in range(N-1):            
        A[k+1:N,k] /= A[k,k]
        for j in range(k+1,N):
            A[k+1:N,j] -= A[k+1:N,k]*A[k,j]
    
    L=tril(A)
    for i in range(N):
        L[i,i]=1.0
    U = triu(A)
    return L, U


L, U = LU(A)
print("2) LU factorization")
print("L = ", L)
print("U = ", U)
print("L*U=",dot(L,U))

2) LU factorization
('L = ', array([[1.0, 0, 0],
       [eps, 1.0, 0],
       [-1, 2*eps/(-eps**2 + 11.6666666666667), 1.0]], dtype=object))
('U = ', array([[1, eps, -1],
       [0, -eps**2 + 11.6666666666667, eps + 1],
       [0, 0, -2*eps*(eps + 1)/(-eps**2 + 11.6666666666667) + 1]], dtype=object))
('L*U=', array([[1.00000000000000, 1.0*eps, -1.00000000000000],
       [eps, 11.6666666666667, 1.00000000000000],
       [-1.00000000000000, eps, 2.00000000000000]], dtype=object))


In [6]:
p=S.Reals
for i in range(1,A.rows+1):
    a=A[0:i,0:i]
    q=solve_poly_inequality(Poly(a.det(),eps),">")
    p=p.intersect(q)

print("3) Gauss factorization LU exists and is unique for epsilon in: ", p)

('3) Gauss factorization LU exists and is unique for epsilon in: ', Interval.open(-7/3, 5/3))


In [7]:
l=Matrix(L).subs({eps:sqrt(35./3)})
u=Matrix(U).subs({eps:sqrt(35./3)})
print("4) LU factorization for epsilon=sqrt(35/3)")
print("L = ", l)
print("U = ", u)
print("L*U=",l*u)


4) LU factorization for epsilon=sqrt(35/3)
('L = ', Matrix([
[             1.0,   0,   0],
[3.41565025531987, 1.0,   0],
[              -1, zoo, 1.0]]))
('U = ', Matrix([
[1, 3.41565025531987,               -1],
[0,                0, 4.41565025531987],
[0,                0,              zoo]]))
('L*U=', Matrix([
[             1.0, 3.41565025531987, nan],
[3.41565025531987, 11.6666666666667, nan],
[             nan,              nan, nan]]))


In [8]:

AS=A.subs({eps:1})
R=AS.cholesky().T
print("5) R upper triangular matrix from cholesky decomposition for epsilon = 1: ",R)
print("R.T * R = ", R.T*R)

('5) R upper triangular matrix from cholesky decomposition for epsilon = 1: ', Matrix([
[1,               1,                -1],
[0, 3.2659863237109, 0.612372435695794],
[0,               0, 0.790569415042095]]))
('R.T * R = ', Matrix([
[ 1,                1,  -1],
[ 1, 11.6666666666667, 1.0],
[-1,              1.0, 2.0]]))
